## 1. Importing Data

In [ ]:
#Alternative way to generate drift.

In [4]:
import pandas as pd

In [5]:
df = pd.read_excel("https://github.com/statisticspoland/ecoicop_classification/blob/master/products_allshops_dataset.xlsx?raw=true")

In [6]:
df

,produkt,kategoria
0,"""słynne roślinne""",Margaryna i inne tłuszcze roślinne
1,#Hejki - Emotki lizaki ręcznie robione o smaka...,Wyroby cukiernicze
2,100% Pur jus d'orange - sok pomarańczowy z mią...,Soki owocowe i warzywne
3,100% sukraloza bez cukru (substancje słodzące),Sztuczne substytuty cukru
4,100% z brzoskwiń produkt owocowy słodzony zag....,"Dżemy, marmolady i miód"
...,...,...
16625,Żywiec Zdrój ze smakiem truskawki Napój niegaz...,Wody mineralne lub źródlane
16626,Żywiec green tea&gruszka,Napoje orzeźwiające
16627,Żywioł - Woda źródlana delikatnie gazowana,Wody mineralne lub źródlane
16628,Żywioł - Woda źródlana gazowana,Wody mineralne lub źródlane


## 2. Generating drift in data

In [7]:
#We drop categories with few rows (less than 50) to streamline the analysis:
df = df[df.groupby("kategoria")["kategoria"].transform("count").ge(50)]
df = df.reset_index(drop=True)

In [8]:
#Let's make two datasets with different distributions of categories
#We'll split the data to reference data and a drifted sample
#The drifted data set can be thought of a as new incoming data for which we want predictions, for example 

In [9]:
drift_sample_size = 50
print(f"There is a total of {len(df)} rows in the dataset")
print(f"Let drift sample size be {drift_sample_size} which leaves {len(df)-drift_sample_size} rows for reference data set")

There is a total of 16263 rows in the dataset
Let drift sample size be 50 which leaves 16213 rows for reference data set


In [10]:
#To simulate a different distribution, let's take the original category distribution, and change the relative size by random value between 0%-x%
original_distribution = df["kategoria"].value_counts(normalize=True, sort=True)
with pd.option_context("display.max_rows", 10):
    display(original_distribution)

Sery i twarogi                                    0.064072
Pozostałe wyroby piekarskie                       0.062719
Warzywa suszone i pozostałe przetwory warzywne    0.053865
Soki owocowe i warzywne                           0.053434
Wyroby cukiernicze                                0.049376
                                                    ...   
Mięso wieprzowe                                   0.003443
Ryby świeże lub chłodzone                         0.003382
Jaja                                              0.003382
Mleko świeże niskotłuszczowe                      0.003320
Mleko świeże pełne                                0.003074
Name: kategoria, Length: 50, dtype: float64

In [11]:
#New distribution:
import random
new_distribution = original_distribution.apply(lambda x: x * random.uniform(1.0, 100.0))
#Normalize to one
new_distribution = new_distribution / new_distribution.sum()
new_distribution = new_distribution.sort_values(ascending = False)
with pd.option_context("display.max_rows", 10):
    display(new_distribution)
print(f"Let's check that the distribution is normalized to one: sum() = {new_distribution.sum()}")

Pozostałe wyroby piekarskie                                   0.109924
Warzywa suszone i pozostałe przetwory warzywne                0.094407
Inne artykuły żywnościowe, gdzie indziej niesklasyfikowane    0.091828
Sól, przyprawy korzenne i zioła kulinarne                     0.054437
Owoce suszone i orzechy                                       0.052185
                                                                ...   
Wędliny                                                       0.000986
Mięso wołowe i cielęce                                        0.000567
Mleko zagęszczone i w proszku                                 0.000501
Mięso drobiowe                                                0.000241
Mleko świeże pełne                                            0.000225
Name: kategoria, Length: 50, dtype: float64

Let's check that the distribution is normalized to one: sum() = 0.9999999999999999


In [12]:
#Now we populate the new dataset
drift_data = df.groupby("kategoria", group_keys=False).apply(lambda x: x.sample(n=int(round(drift_sample_size*new_distribution[x.name])))).copy()
display(drift_data["kategoria"].value_counts(normalize=True, sort=True))
print(f"Length: {len(drift_data)}")

Warzywa suszone i pozostałe przetwory warzywne                                     0.111111
Inne artykuły żywnościowe, gdzie indziej niesklasyfikowane                         0.111111
Pozostałe wyroby piekarskie                                                        0.111111
Sól, przyprawy korzenne i zioła kulinarne                                          0.066667
Owoce suszone i orzechy                                                            0.066667
Pozostałe przetwory z ryb i owoców morza                                           0.044444
Napoje orzeźwiające                                                                0.044444
Sosy, przyprawy                                                                    0.044444
Żywność dla dzieci                                                                 0.044444
Makarony i produkty makaronowe                                                     0.044444
Jogurt                                                                          

Length: 45


In [13]:
#Lets remove the drift data rows from reference set
print(len(df))
reference_data = df.drop(drift_data.index)

16263
